In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163345 sha256=74d9addfe40f748255d7e4c2d70a4ab8e5209110254f2a071f998f8223a85caf
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
import pandas as pd
import numpy as np
import csv
from surprise import Reader, Dataset, KNNBasic
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt


In [3]:
# from google.colab import files
# uploaded = files.upload()
# import io
# df1=pd.read_csv(io.BytesIO(uploaded['shortened.csv']))
# df1
df = pd.read_csv('Processed_Book_Data_shortened.csv')
df.head()

,0,user_id,isbn,rating,book_title,book_author,publisher,img_s,Summary,Language,Category
0,0,2,195153448,0,Classical Mythology,Mark P. O. Morford,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,Provides an introduction to classical myths pl...,en,['Social Science']
1,1,8,2005018,5,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses']
2,2,11400,2005018,0,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses']
3,3,11676,2005018,8,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses']
4,4,41385,2005018,0,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses']


In [4]:
df.shape

(258879, 11)

In [5]:
from surprise.accuracy import rmse
from surprise.model_selection import  train_test_split
reader = Reader()
data = Dataset.load_from_df(df[['user_id', 'isbn', 'rating']], reader)
train, test = train_test_split(data, test_size=0.25, random_state=1)

Trying different number of neighours to see if this parameter affects the performance of the model.  
The default number of factors is 40, hence we will tweak the number of neighours from 1-100 to evaluate the performance.

In [5]:
import time
from collections import defaultdict

# A dictionary to store the rsme, fit time and prediction time 
knn_dict = defaultdict(dict)
for i in range(100):
    knn = KNNBasic(k=i+1)
    start_time = time.time()
    knn.fit(train)
    fit_time = time.time() - start_time
    predictions = knn.test(test)
    knn_dict[i+1]['rmse'] = rmse(predictions)
    knn_dict[i+1]['fit_time'] = fit_time
    knn_dict[i+1]['prediction_time'] = time.time() - start_time - fit_time

# A dataframe to store the results
knn_df = pd.DataFrame(knn_dict)
knn_df

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 4.2328
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 4.1487
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 4.0908
Computing the msd similarity matrix...
Done computing similarity matrix.


KeyboardInterrupt: 

We will be using the default parameters set in the KNNBasic for our comparison between the different algorithms.

In [6]:
import time
KNN = KNNBasic()
begin_fit = time.time()
KNN.fit(train)
end_fit = time.time()
begin_predict = time.time()
KNN.test(test)
end_predict = time.time()
result = rmse(KNN.test(test))
row = {"Algorithm":"KNN","Fit Time":end_fit-begin_fit,"Prediction Time":end_predict-begin_predict}
print(row)

Computing the msd similarity matrix...


In [ ]:
testdf = pd.DataFrame(test)
testdf = testdf.rename(columns={
    0: "User-ID",
    1: "isbn",
    2: "Actual Rating"
})
testdf["Predicted Rating"] = -1
testdf["Error"] = -1
for index, row in testdf.iterrows():
  testdf.loc[index, 'Predicted Rating'] = KNN.predict(row["User-ID"], row["isbn"], row["Actual Rating"]).est
for index, row in testdf.iterrows():
  testdf.loc[index, 'Error'] = '{:.2f}'.format(abs(row["Predicted Rating"] - row["Actual Rating"]))

testdf

In [ ]:
uniqueusers = len(pd.unique(df['User-ID']))
uniquebooks = len(pd.unique(df["ISBN"]))
print(uniqueusers,uniquebooks)

In [ ]:
worst_predictions = testdf.sort_values(by='Error',ascending=False)[:50]
best_predictions = testdf.sort_values(by='Error',ascending=False)[-50:]

best_predictions

In [ ]:
counts1 = df['user_id'].value_counts()
counts2 = df['isbn'].value_counts()

df = df[~df['user_id'].isin(counts1[counts1 < 200].index)]
df = df[~df['isbn'].isin(counts2[counts2 < 100].index)]

df = df.drop_duplicates(["book_title", "user_id"])

piv = df.pivot(index='book_title', columns='user_id', values='rating').fillna(0)

In [ ]:
piv

user_id,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271284,271448,271705,273979,274004,274061,274308,275970,277427,278418
book_title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Without Remorse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
matrix = piv.values

In [ ]:

matrix.shape

(673, 598)

In [ ]:
from sklearn.neighbors import NearestNeighbors
model_knn=NearestNeighbors(metric='cosine',algorithm='brute')
model_knn.fit(matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  x=piv.loc[book].array.reshape(1, -1)
  distances,indices=model_knn.kneighbors(x,n_neighbors=6)
  R_books=[]
  for distance,indice in zip(distances[0],indices[0]):
    if distance!=0:
      R_book=piv.index[indice]
      R_books.append([R_book,distance])
  recommended_books=[book,R_books[::-1]]
  return recommended_books

In [ ]:
get_recommends('The Witching Hour (Lives of the Mayfair Witches)')

['The Witching Hour (Lives of the Mayfair Witches)',
 [['Eaters of the Dead', 0.7133479788795423],
  ['The Tale of the Body Thief (Vampire Chronicles (Paperback))',
   0.690940306509213],
  ['Interview with the Vampire', 0.6611868109275274],
  ['Catch 22', 0.6492703211363626],
  ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.6432487145987553],
  ['The Witching Hour (Lives of the Mayfair Witches)',
   1.1102230246251565e-16]]]

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.7549828220814179], ['The Surgeon', 0.7469825938212555], ['I Know This Much Is True', 0.7432840478726268], ['The Lovely Bones: A Novel', 0.7335090730124778], ["The Dive From Clausen's Pier (Alex Awards)", 0.7182736946652024]]]
You passed the challenge! 🎉🎉🎉🎉🎉
